In [35]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [37]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [38]:
import os
import pandas as pd
from datetime import datetime, timezone

from configs import config
from utils.custom_classes.data_loader import CompasWithoutSensitiveAttrsDataset
from utils.metrics_computation_interfaces import compute_metrics_multiple_runs

## Configs

In [39]:
DATASET_NAME = "COMPAS_Without_Sensitive_Attributes"

# RUNS_SEED_LST = [101, 111, 201, 221, 311, 333, 410, 555, 666, 851, 999, 1010]
RUNS_SEED_LST = [100, 200]
SEX_priv = 0
RACE_priv = 'Caucasian'
N_ESTIMATORS = 20
SENSITIVE_ATTRIBUTES = ['sex', 'race']
PRIV_VALUES = [SEX_priv, RACE_priv]
TUNED_PARAMS_FILE_PATH = os.path.join('results', 'models_tuning', f'tuning_results_{DATASET_NAME}_20230117__012024.csv')
SAVE_RESULTS_DIR_PATH = os.path.join('results', 'hypothesis_space', f'{DATASET_NAME}_Metrics_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}')

## Models tuned hyper-parameters

In [40]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,COMPAS_Without_Sensitive_Attributes,LogisticRegression,0.6785,0.6837,"{'C': 1, 'max_iter': 50, 'penalty': 'l2', 'sol..."
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,0.6952,0.6989,"{'criterion': 'gini', 'max_depth': 20, 'max_fe..."
2,COMPAS_Without_Sensitive_Attributes,RandomForestClassifier,0.6962,0.7008,"{'max_depth': 4, 'max_features': 0.6, 'min_sam..."
3,COMPAS_Without_Sensitive_Attributes,XGBClassifier,0.6971,0.7008,"{'lambda': 100, 'learning_rate': 0.1, 'max_dep..."
4,COMPAS_Without_Sensitive_Attributes,KNeighborsClassifier,0.7056,0.7112,"{'metric': 'minkowski', 'n_neighbors': 25, 'we..."
5,COMPAS_Without_Sensitive_Attributes,MLPClassifier,0.6834,0.6875,"{'activation': 'relu', 'hidden_layer_sizes': (..."


## Load dataset

In [41]:
dataset = CompasWithoutSensitiveAttrsDataset(dataset_path='data/COMPAS.csv')
dataset.X_data.head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M
0,0.0,-2.340451,1.0,-15.010999,1,0,0,0,1
1,0.0,0.000000,0.0,0.000000,1,0,0,1,0
2,0.0,0.000000,0.0,0.000000,0,0,1,1,0
3,0.0,0.000000,0.0,6.000000,1,0,0,0,1
4,0.0,0.000000,0.0,7.513697,1,0,0,1,0


## Run experiments

In [42]:
multiple_runs_metrics_dct = compute_metrics_multiple_runs(dataset, config.TEST_SET_FRACTION, DATASET_NAME,
                                                          config, models_tuned_params_df, N_ESTIMATORS, SENSITIVE_ATTRIBUTES, PRIV_VALUES,
                                                          RUNS_SEED_LST, SAVE_RESULTS_DIR_PATH)

  0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze DecisionTreeClassifier  ##############################
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)

Protected groups splits:
sex_race_priv (88, 11)
sex_race_dis (518, 11)
sex_priv (199, 11)
sex_dis (857, 11)
race_priv (427, 11)
race_dis (629, 11)


X train and validation set: 


,age_cat_25 - 45_0,c_charge_degree_M_1,juv_other_count,age_cat_Less than 25_0,c_charge_degree_F_0,c_charge_degree_F_1,age_cat_Less than 25_1,age_cat_25 - 45_1,juv_fel_count,juv_misd_count,priors_count,c_charge_degree_M_0,age_cat_Greater than 45_0,age_cat_Greater than 45_1
470,0,0,-0.148179,1,0,1,0,1,-0.098697,-0.127240,0.106154,1,1,0
1328,0,0,-0.148179,1,0,1,0,1,-0.098697,0.348324,2.409132,1,1,0
3551,0,1,-0.148179,1,1,0,0,1,-0.098697,-0.127240,0.620185,0,1,0
1628,0,1,-0.148179,1,1,0,0,1,-0.098697,-0.127240,-0.085761,0,1,0
4138,0,0,1.111816,1,0,1,0,1,-0.098697,-0.127240,-0.280851,1,1,0
5030,0,0,-0.148179,1,0,1,0,1,-0.098697,-0.127240,0.693041,1,1,0
4510,0,1,-0.148179,1,1,0,0,1,-0.098697,-0.127240,-0.085761,0,1,0
1676,1,0,-0.148179,0,0,1,1,0,-0.098697,-0.127240,-0.661506,1,1,0
5047,0,1,-0.148179,1,1,0,0,1,-0.098697,-0.127240,-0.469591,0,1,0
1554,1,1,-0.148179,1,1,0,0,0,-0.098697,-0.127240,-0.668317,0,0,1


2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 20
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 20 was tested
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 20
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 20 was tested
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 20
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 20 was tested
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 20
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 20 was tested
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 20
2023-01-22 01:59:39 abstract_overall_variance_analyzer.py INFO    : Classifier 5 / 20 was tested




##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.6818
Mean: 0.527
Std: 0.073
IQR: 0.0854
Entropy: 0.0
Jitter: 0.1427
Per sample accuracy: 0.6588
Label stability: 0.8114



[DecisionTreeClassifier] Metrics confusion matrix:


...

,age_cat_25 - 45_0,c_charge_degree_M_1,juv_other_count,age_cat_Less than 25_0,c_charge_degree_F_0,c_charge_degree_F_1,age_cat_Less than 25_1,age_cat_25 - 45_1,juv_fel_count,juv_misd_count,priors_count,c_charge_degree_M_0,age_cat_Greater than 45_0,age_cat_Greater than 45_1
2780,0,0,-0.157934,1,0,1,0,1,-0.090147,-0.131370,2.962771,1,1,0
1047,0,1,-0.157934,1,1,0,0,1,-0.090147,-0.131370,0.106375,0,1,0
3152,1,0,-0.157934,1,0,1,0,0,-0.090147,-0.131370,2.440022,1,0,1
2076,1,0,-0.157934,0,0,1,1,0,-0.090147,1.035096,-0.274477,1,1,0
1015,1,1,-0.157934,1,1,0,0,0,-0.090147,-0.131370,1.635749,0,0,1
3981,0,0,-0.157934,1,0,1,0,1,-0.090147,-0.131370,-0.655330,1,1,0
1171,1,0,3.909141,0,0,1,1,0,-0.090147,0.261117,0.015578,1,1,0
2653,0,1,-0.157934,1,1,0,0,1,-0.090147,-0.131370,-0.655330,0,1,0
4567,1,0,-0.157934,1,0,1,0,0,-0.090147,-0.131370,0.487228,1,0,1
2102,0,0,-0.157934,1,0,1,0,1,-0.090147,-0.131370,-0.655330,1,1,0


2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 20
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 20 was tested
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 20
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 20 was tested
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 20
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 20 was tested
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 20
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 20 was tested
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 20
2023-01-22 01:59:41 abstract_overall_variance_analyzer.py INFO    : Classifier 5 / 20 was tested




##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.6695
Mean: 0.5227
Std: 0.0708
IQR: 0.0843
Entropy: 0.0
Jitter: 0.1211
Per sample accuracy: 0.6624
Label stability: 0.8501



[DecisionTreeClassifier] Metrics confusion matrix:


,Metric,overall,sex_race_priv,sex_race_dis,sex_priv,sex_dis,race_priv,race_dis,Model_Seed,Model_Name
0,General_Ensemble_Accuracy,0.669508,0.701149,0.674157,0.680000,0.667056,0.665037,0.672334,201,DecisionTreeClassifier
1,Mean,0.522703,0.601949,0.471472,0.558899,0.514245,0.588747,0.480953,201,DecisionTreeClassifier
2,Std,0.070803,0.080199,0.072303,0.077259,0.069295,0.067687,0.072773,201,DecisionTreeClassifier
3,IQR,0.084336,0.104779,0.083739,0.094597,0.081939,0.084446,0.084267,201,DecisionTreeClassifier
4,Entropy,0.000000,0.000000,0.000000,0.199215,0.000000,0.000000,0.000000,201,DecisionTreeClassifier
5,Jitter,0.121117,0.133031,0.129489,0.133842,0.118144,0.106499,0.130359,201,DecisionTreeClassifier
6,Per_Sample_Accuracy,0.662358,0.677586,0.663670,0.665000,0.661741,0.662592,0.662210,201,DecisionTreeClassifier
7,Label_Stability,0.850095,0.819540,0.841573,0.824000,0.856192,0.867482,0.839104,201,DecisionTreeClassifier
8,TPR,0.637895,0.531250,0.720848,0.610390,0.643216,0.469388,0.713415,201,DecisionTreeClassifier
9,TNR,0.695353,0.800000,0.621514,0.723577,0.687773,0.774809,0.630094,201,DecisionTreeClassifier


100%|██████████| 1/1 [00:02<00:00,  2.38s/it]